In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH11-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH11-Retriever


In [3]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [4]:
loaders = [
    TextLoader("./data/appendix-keywords.txt", encoding="utf-8")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

# 전체 문서 검색

In [5]:
# 자식 분할기를 생성
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# DB를 생성
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings(),
)

store = InMemoryStore()

# Retriever를 생성
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    child_splitter=child_splitter,
    docstore=store,
)

In [6]:
# 문서를 검색기에 추가. docs는 문서 목록, ids는 문서의 고유 식별자 목록
retriever.add_documents(docs, ids=None, add_to_docstore=True)

In [7]:
# 저장소의 모든 키를 리스트로 반환
list(store.yield_keys())

['39fed4c5-21be-482e-a4d9-f5b3d8acf7be']

In [8]:
# 유사도 검색을 실행
sub_docs = vectorstore.similarity_search("Word2Vec")

In [9]:
# sub_docs 리스트의 첫 번째 요소의  page_content 속성을 출력
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


In [10]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Word2Vec")

In [11]:
# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 5733


 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을


# 더 큰 Chunk 의 크기를 조절

In [12]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

# 자식 문서를 생성하는데 사용되는 텍스트 분할기
# 부모보다 작은 문서를 생성
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# 자식 청크를 인덱싱하는데 사용할 벡터 저장소
vectorstore = Chroma(
    collection_name="split_parent",
    embedding_function=OpenAIEmbeddings(),
)

# 부모 문서의 저장 계층
store = InMemoryStore()

In [13]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    child_splitter=child_splitter,
    docstore=store,
    parent_splitter=parent_splitter,
)

In [14]:
retriever.add_documents(docs)

In [15]:
# 저장소에서 키를 생성하고 리스트로 변환한 후 길이를 반환
len(list(store.yield_keys()))

7

In [16]:
# 유사도 검색을 수행
sub_docs = vectorstore.similarity_search("Word2Vec")


In [17]:
# sub_docs 리스트이 첫 번째 요소의 page_content 속성을 출력
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


In [18]:
# 문서를 검새갛여 가져옴
retrieved_docs = retriever.invoke("Word2Vec")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환
print(retrieved_docs[0].page_content)

정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.
예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.
연관키워드: 딥러닝, 자연어 처리, Attention

HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
